In [1]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import re
import tensorflow as tf
import numpy as np

# Tracking vidéo
En partant d'une première détection de l'objet d'intérêt qui prend la forme d'une boite englobante que l'on suppose correcte, on va suivre l'objet sur les frames suivantes.

Pour chaque frame, on prend la boîte englobante de la frame précédente, on va créer un set de nouvelles boîtes potentielles. On va ensuite récupérer le "patch" de l'image associé à chaque nouvelle boîte englobante et effectuer une prediction à l'aide du modèle précédemment créé. On conservera comme nouvelle boîte celle qui fournira la prédiction la plus précise (pourcentage le plus élevé).

In [2]:
def display_images(images, lab, n_line, l_col):
    """ Function used to display mutliple image with pyplot
    """
    plt.figure(figsize=(10, 10))
    for i in range(min(n_line*l_col, len(images))):
        ax = plt.subplot(n_line, l_col, i + 1)
        plt.imshow(images[i])
        plt.title(f'({lab[i]:.2})')
        plt.axis("off")

In [68]:
def build_boxes(x, y, w, h, N, stride, W, H):
    """ Build a set of new potential bouding boxes from a given box
    """
    # N = 15
    # stride = 1
    shift = list(range(-N, N+1, stride))
    zoom = [1]
    
    pos = []
    for i in shift:
        new_x = x + i
        
        for j in shift:
            new_y = y + j
            
            new_pos = (new_x, new_y, w, h)
            pos.append(new_pos)
            
            for z in zoom:
                new_w, new_h = int(w*z), int(h*z)
                new_x = new_x - (abs(new_w - w) // 2)
                new_y = new_y - (abs(new_h - h) // 2)
                
                new_x = min(max(0, new_x), W)
                new_y = min(max(0, new_y), H)
                
                new_w = min((W - new_x), new_w)
                new_h = min((H - new_y), new_h)
                
                pos.append((new_x, new_y, new_w, new_h))
            
    return pos

In [69]:
def read_bounding_box(path):
    """ Read the bouding box file of a video
    """
    boxes = {}
    m = re.search('VIDEOS/(.+)\.mp4', path)
    filename = m.group(1)
    
    f = open(f'../GT/{filename}_2_bboxes.txt', 'r')
    lines = f.readlines()
    
    for l in lines:
        elmts = l.split(' ')
        frame = int(elmts[0])
        
        if elmts[1] == '1':
            boxes[frame] = (int(elmts[2]), int(elmts[3]), int(elmts[4]), int(elmts[5]))
            
    return boxes

In [70]:
model = tf.keras.models.load_model('../models/best_models', custom_objects=None, compile=True, options=None)

In [71]:
labels = { 0: 'bol', 1: 'coca', 2: 'lait', 3: 'riz', 4: 'sucre' }

def get_images(filename):
    predict_boxes = {}
    
    # on récupère les boundings box du fichier
    boxes = read_bounding_box(filename)
    
    # on ouvre le fichier
    cap = cv2.VideoCapture(filename)
    nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    if(not cap.isOpened()):
        print("ERROR: unable to read video:", video_filename)
        sys.exit()
    
    print(f'Loading {nframes} frames')
    print(len(boxes))
    
    if 'CanOfCocaCola' in filename:
        label = 1
    elif 'MilkBottle' in filename:
        label = 2
    elif 'Bowl' in filename:
        label = 0
    elif 'Rice' in filename:
        label = 3
    elif 'Sugar' in filename:
        label = 4
    
    foundObj = False
    for f in range(nframes):
    
        ret, frame = cap.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # conversion BRG to RGB
        
        # kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
        # frame = cv2.filter2D(frame, -1, kernel)
        
        f_width, f_height, _ = frame.shape
        
        if f in boxes: # on ne regarde que les frames qui ont une bounding box
            #print(f)
            if not foundObj:
                box = boxes[f] # première bounding box (= box de référence)
            else:
                #print(f'a_{f}')
                x, y, w, h = box
                
                # on créé de nouvelles bounding box autour de celle de la frame précédente
                new_boxes = build_boxes(x, y, w, h, 20, 3, f_width, f_height)
                
                # print(f'BOXES = {len(new_boxes)}')
                #patch = frame[y:y+h, x:x+w]
                print(f'BOX = {box}')
                
                # pour chaque nouvel box, on extrait le patch associé dans l'image
                patches = np.zeros((0, 227, 227, 3))
                for bb in new_boxes:
                    x1, y1, w1, h1 = bb
                    print(bb)
                    
                    new_patch = frame[y1:y1+h1, x1:x1+w1, :]
                    print(new_patch.shape)
                    if 0 not in new_patch.shape:
                        new_patch = tf.image.resize(new_patch, (227, 227), method='nearest')
                        patches = tf.concat((patches, tf.expand_dims(new_patch, 0)), axis=0)
                
                # on passe les patchs dans le modèle
                if patches.shape[0] > 0:
                    #print(f'b_{f}')
                    pred = model.predict(patches / 255)
                    
                    # on récupère la meilleure prédiction
                    best_pred = np.argmax(pred[:, label])
                    
                    #display_images(patches, pred[:, label], 10, 10)
                    #break

                    # on récupère le patch associé à la meilleure prédiction
                    best_patch = patches[best_pred]
                    
                    #if f == 100:
                    #display_images(patches, pred[:, label], 7, 7)
                    plt.imshow(best_patch)
                    plt.axis("off")
                    #print(pred[best_pred])
                    #print(pred[best_pred, label])
                    plt.savefig(f'img/frame__best{f}.png')
                    
                    # break
                    print(pred[best_pred, label])
                        
                    best_box = new_boxes[best_pred]

                    # on modifie la current box pour la frame suivante
                    box = best_box
                    predict_boxes[f] = best_box
                    
            foundObj = True
    
    return predict_boxes

In [ ]:
tracking = get_images('../VIDEOS/RicePlace3Subject3.mp4')

Loading 226 frames
164
BOX = (864, 344, 156, 222)
(844, 324, 156, 222)
(222, 156, 3)
(844, 324, 156, 222)
(222, 156, 3)
(844, 327, 156, 222)
(222, 156, 3)
(844, 327, 156, 222)
(222, 156, 3)
(844, 330, 156, 222)
(222, 156, 3)
(844, 330, 156, 222)
(222, 156, 3)
(844, 333, 156, 222)
(222, 156, 3)
(844, 333, 156, 222)
(222, 156, 3)
(844, 336, 156, 222)
(222, 156, 3)
(844, 336, 156, 222)
(222, 156, 3)
(844, 339, 156, 222)
(222, 156, 3)
(844, 339, 156, 222)
(222, 156, 3)
(844, 342, 156, 222)
(222, 156, 3)
(844, 342, 156, 222)
(222, 156, 3)
(844, 345, 156, 222)
(222, 156, 3)
(844, 345, 156, 222)
(222, 156, 3)
(844, 348, 156, 222)
(222, 156, 3)
(844, 348, 156, 222)
(222, 156, 3)
(844, 351, 156, 222)
(222, 156, 3)
(844, 351, 156, 222)
(222, 156, 3)
(844, 354, 156, 222)
(222, 156, 3)
(844, 354, 156, 222)
(222, 156, 3)
(844, 357, 156, 222)
(222, 156, 3)
(844, 357, 156, 222)
(222, 156, 3)
(844, 360, 156, 222)
(222, 156, 3)
(844, 360, 156, 222)
(222, 156, 3)
(844, 363, 156, 222)
(222, 156, 3)
(844,

(868, 342, 156, 222)
(222, 156, 3)
(868, 345, 156, 222)
(222, 156, 3)
(868, 345, 156, 222)
(222, 156, 3)
(868, 348, 156, 222)
(222, 156, 3)
(868, 348, 156, 222)
(222, 156, 3)
(868, 351, 156, 222)
(222, 156, 3)
(868, 351, 156, 222)
(222, 156, 3)
(868, 354, 156, 222)
(222, 156, 3)
(868, 354, 156, 222)
(222, 156, 3)
(868, 357, 156, 222)
(222, 156, 3)
(868, 357, 156, 222)
(222, 156, 3)
(868, 360, 156, 222)
(222, 156, 3)
(868, 360, 156, 222)
(222, 156, 3)
(868, 363, 156, 222)
(222, 156, 3)
(868, 363, 156, 222)
(222, 156, 3)
(871, 324, 156, 222)
(222, 156, 3)
(871, 324, 156, 222)
(222, 156, 3)
(871, 327, 156, 222)
(222, 156, 3)
(871, 327, 156, 222)
(222, 156, 3)
(871, 330, 156, 222)
(222, 156, 3)
(871, 330, 156, 222)
(222, 156, 3)
(871, 333, 156, 222)
(222, 156, 3)
(871, 333, 156, 222)
(222, 156, 3)
(871, 336, 156, 222)
(222, 156, 3)
(871, 336, 156, 222)
(222, 156, 3)
(871, 339, 156, 222)
(222, 156, 3)
(871, 339, 156, 222)
(222, 156, 3)
(871, 342, 156, 222)
(222, 156, 3)
(871, 342, 156, 222)

(872, 316, 156, 222)
(222, 156, 3)
(872, 316, 156, 222)
(222, 156, 3)
(872, 319, 156, 222)
(222, 156, 3)
(872, 319, 156, 222)
(222, 156, 3)
(872, 322, 156, 222)
(222, 156, 3)
(872, 322, 156, 222)
(222, 156, 3)
(872, 325, 156, 222)
(222, 156, 3)
(872, 325, 156, 222)
(222, 156, 3)
(872, 328, 156, 222)
(222, 156, 3)
(872, 328, 156, 222)
(222, 156, 3)
(872, 331, 156, 222)
(222, 156, 3)
(872, 331, 156, 222)
(222, 156, 3)
(872, 334, 156, 222)
(222, 156, 3)
(872, 334, 156, 222)
(222, 156, 3)
(872, 337, 156, 222)
(222, 156, 3)
(872, 337, 156, 222)
(222, 156, 3)
(872, 340, 156, 222)
(222, 156, 3)
(872, 340, 156, 222)
(222, 156, 3)
(872, 343, 156, 222)
(222, 156, 3)
(872, 343, 156, 222)
(222, 156, 3)
(875, 304, 156, 222)
(222, 156, 3)
(875, 304, 156, 222)
(222, 156, 3)
(875, 307, 156, 222)
(222, 156, 3)
(875, 307, 156, 222)
(222, 156, 3)
(875, 310, 156, 222)
(222, 156, 3)
(875, 310, 156, 222)
(222, 156, 3)
(875, 313, 156, 222)
(222, 156, 3)
(875, 313, 156, 222)
(222, 156, 3)
(875, 316, 156, 222)

(896, 343, 156, 222)
(222, 156, 3)
(899, 304, 156, 222)
(222, 156, 3)
(899, 304, 156, 222)
(222, 156, 3)
(899, 307, 156, 222)
(222, 156, 3)
(899, 307, 156, 222)
(222, 156, 3)
(899, 310, 156, 222)
(222, 156, 3)
(899, 310, 156, 222)
(222, 156, 3)
(899, 313, 156, 222)
(222, 156, 3)
(899, 313, 156, 222)
(222, 156, 3)
(899, 316, 156, 222)
(222, 156, 3)
(899, 316, 156, 222)
(222, 156, 3)
(899, 319, 156, 222)
(222, 156, 3)
(899, 319, 156, 222)
(222, 156, 3)
(899, 322, 156, 222)
(222, 156, 3)
(899, 322, 156, 222)
(222, 156, 3)
(899, 325, 156, 222)
(222, 156, 3)
(899, 325, 156, 222)
(222, 156, 3)
(899, 328, 156, 222)
(222, 156, 3)
(899, 328, 156, 222)
(222, 156, 3)
(899, 331, 156, 222)
(222, 156, 3)
(899, 331, 156, 222)
(222, 156, 3)
(899, 334, 156, 222)
(222, 156, 3)
(899, 334, 156, 222)
(222, 156, 3)
(899, 337, 156, 222)
(222, 156, 3)
(899, 337, 156, 222)
(222, 156, 3)
(899, 340, 156, 222)
(222, 156, 3)
(899, 340, 156, 222)
(222, 156, 3)
(899, 343, 156, 222)
(222, 156, 3)
(899, 343, 156, 222)

(897, 317, 156, 222)
(222, 156, 3)
(897, 317, 156, 222)
(222, 156, 3)
(897, 320, 156, 222)
(222, 156, 3)
(897, 320, 156, 222)
(222, 156, 3)
(897, 323, 156, 222)
(222, 156, 3)
(897, 323, 156, 222)
(222, 156, 3)
(897, 326, 156, 222)
(222, 156, 3)
(897, 326, 156, 222)
(222, 156, 3)
(897, 329, 156, 222)
(222, 156, 3)
(897, 329, 156, 222)
(222, 156, 3)
(897, 332, 156, 222)
(222, 156, 3)
(897, 332, 156, 222)
(222, 156, 3)
(897, 335, 156, 222)
(222, 156, 3)
(897, 335, 156, 222)
(222, 156, 3)
(897, 338, 156, 222)
(222, 156, 3)
(897, 338, 156, 222)
(222, 156, 3)
(900, 299, 156, 222)
(222, 156, 3)
(900, 299, 156, 222)
(222, 156, 3)
(900, 302, 156, 222)
(222, 156, 3)
(900, 302, 156, 222)
(222, 156, 3)
(900, 305, 156, 222)
(222, 156, 3)
(900, 305, 156, 222)
(222, 156, 3)
(900, 308, 156, 222)
(222, 156, 3)
(900, 308, 156, 222)
(222, 156, 3)
(900, 311, 156, 222)
(222, 156, 3)
(900, 311, 156, 222)
(222, 156, 3)
(900, 314, 156, 222)
(222, 156, 3)
(900, 314, 156, 222)
(222, 156, 3)
(900, 317, 156, 222)

...

(878, 658, 156, 222)
(222, 156, 3)
(878, 661, 156, 222)
(222, 156, 3)
(878, 661, 156, 222)
(222, 156, 3)
(878, 664, 156, 222)
(222, 156, 3)
(878, 664, 156, 222)
(222, 156, 3)
1.0
BOX = (857, 646, 156, 222)
(837, 626, 156, 222)
(222, 156, 3)
(837, 626, 156, 222)
(222, 156, 3)
(837, 629, 156, 222)
(222, 156, 3)
(837, 629, 156, 222)
(222, 156, 3)
(837, 632, 156, 222)
(222, 156, 3)
(837, 632, 156, 222)
(222, 156, 3)
(837, 635, 156, 222)
(222, 156, 3)
(837, 635, 156, 222)
(222, 156, 3)
(837, 638, 156, 222)
(222, 156, 3)
(837, 638, 156, 222)
(222, 156, 3)
(837, 641, 156, 222)
(222, 156, 3)
(837, 641, 156, 222)
(222, 156, 3)
(837, 644, 156, 222)
(222, 156, 3)
(837, 644, 156, 222)
(222, 156, 3)
(837, 647, 156, 222)
(222, 156, 3)
(837, 647, 156, 222)
(222, 156, 3)
(837, 650, 156, 222)
(222, 156, 3)
(837, 650, 156, 222)
(222, 156, 3)
(837, 653, 156, 222)
(222, 156, 3)
(837, 653, 156, 222)
(222, 156, 3)
(837, 656, 156, 222)
(222, 156, 3)
(837, 656, 156, 222)
(222, 156, 3)
(837, 659, 156, 222)
(22

(861, 644, 156, 222)
(222, 156, 3)
(861, 647, 156, 222)
(222, 156, 3)
(861, 647, 156, 222)
(222, 156, 3)
(861, 650, 156, 222)
(222, 156, 3)
(861, 650, 156, 222)
(222, 156, 3)
(861, 653, 156, 222)
(222, 156, 3)
(861, 653, 156, 222)
(222, 156, 3)
(861, 656, 156, 222)
(222, 156, 3)
(861, 656, 156, 222)
(222, 156, 3)
(861, 659, 156, 222)
(222, 156, 3)
(861, 659, 156, 222)
(222, 156, 3)
(861, 662, 156, 222)
(222, 156, 3)
(861, 662, 156, 222)
(222, 156, 3)
(861, 665, 156, 222)
(222, 156, 3)
(861, 665, 156, 222)
(222, 156, 3)
(864, 626, 156, 222)
(222, 156, 3)
(864, 626, 156, 222)
(222, 156, 3)
(864, 629, 156, 222)
(222, 156, 3)
(864, 629, 156, 222)
(222, 156, 3)
(864, 632, 156, 222)
(222, 156, 3)
(864, 632, 156, 222)
(222, 156, 3)
(864, 635, 156, 222)
(222, 156, 3)
(864, 635, 156, 222)
(222, 156, 3)
(864, 638, 156, 222)
(222, 156, 3)
(864, 638, 156, 222)
(222, 156, 3)
(864, 641, 156, 222)
(222, 156, 3)
(864, 641, 156, 222)
(222, 156, 3)
(864, 644, 156, 222)
(222, 156, 3)
(864, 644, 156, 222)

(844, 642, 156, 222)
(222, 156, 3)
(844, 645, 156, 222)
(222, 156, 3)
(844, 645, 156, 222)
(222, 156, 3)
(844, 648, 156, 222)
(222, 156, 3)
(844, 648, 156, 222)
(222, 156, 3)
(844, 651, 156, 222)
(222, 156, 3)
(844, 651, 156, 222)
(222, 156, 3)
(844, 654, 156, 222)
(222, 156, 3)
(844, 654, 156, 222)
(222, 156, 3)
(844, 657, 156, 222)
(222, 156, 3)
(844, 657, 156, 222)
(222, 156, 3)
(844, 660, 156, 222)
(222, 156, 3)
(844, 660, 156, 222)
(222, 156, 3)
(847, 621, 156, 222)
(222, 156, 3)
(847, 621, 156, 222)
(222, 156, 3)
(847, 624, 156, 222)
(222, 156, 3)
(847, 624, 156, 222)
(222, 156, 3)
(847, 627, 156, 222)
(222, 156, 3)
(847, 627, 156, 222)
(222, 156, 3)
(847, 630, 156, 222)
(222, 156, 3)
(847, 630, 156, 222)
(222, 156, 3)
(847, 633, 156, 222)
(222, 156, 3)
(847, 633, 156, 222)
(222, 156, 3)
(847, 636, 156, 222)
(222, 156, 3)
(847, 636, 156, 222)
(222, 156, 3)
(847, 639, 156, 222)
(222, 156, 3)
(847, 639, 156, 222)
(222, 156, 3)
(847, 642, 156, 222)
(222, 156, 3)
(847, 642, 156, 222)

(222, 156, 3)
(871, 624, 156, 222)
(222, 156, 3)
(871, 627, 156, 222)
(222, 156, 3)
(871, 627, 156, 222)
(222, 156, 3)
(871, 630, 156, 222)
(222, 156, 3)
(871, 630, 156, 222)
(222, 156, 3)
(871, 633, 156, 222)
(222, 156, 3)
(871, 633, 156, 222)
(222, 156, 3)
(871, 636, 156, 222)
(222, 156, 3)
(871, 636, 156, 222)
(222, 156, 3)
(871, 639, 156, 222)
(222, 156, 3)
(871, 639, 156, 222)
(222, 156, 3)
(871, 642, 156, 222)
(222, 156, 3)
(871, 642, 156, 222)
(222, 156, 3)
(871, 645, 156, 222)
(222, 156, 3)
(871, 645, 156, 222)
(222, 156, 3)
(871, 648, 156, 222)
(222, 156, 3)
(871, 648, 156, 222)
(222, 156, 3)
(871, 651, 156, 222)
(222, 156, 3)
(871, 651, 156, 222)
(222, 156, 3)
(871, 654, 156, 222)
(222, 156, 3)
(871, 654, 156, 222)
(222, 156, 3)
(871, 657, 156, 222)
(222, 156, 3)
(871, 657, 156, 222)
(222, 156, 3)
(871, 660, 156, 222)
(222, 156, 3)
(871, 660, 156, 222)
(222, 156, 3)
(874, 621, 156, 222)
(222, 156, 3)
(874, 621, 156, 222)
(222, 156, 3)
(874, 624, 156, 222)
(222, 156, 3)
(874, 

(851, 634, 156, 222)
(222, 156, 3)
(851, 637, 156, 222)
(222, 156, 3)
(851, 637, 156, 222)
(222, 156, 3)
(851, 640, 156, 222)
(222, 156, 3)
(851, 640, 156, 222)
(222, 156, 3)
(851, 643, 156, 222)
(222, 156, 3)
(851, 643, 156, 222)
(222, 156, 3)
(851, 646, 156, 222)
(222, 156, 3)
(851, 646, 156, 222)
(222, 156, 3)
(851, 649, 156, 222)
(222, 156, 3)
(851, 649, 156, 222)
(222, 156, 3)
(851, 652, 156, 222)
(222, 156, 3)
(851, 652, 156, 222)
(222, 156, 3)
(851, 655, 156, 222)
(222, 156, 3)
(851, 655, 156, 222)
(222, 156, 3)
(854, 616, 156, 222)
(222, 156, 3)
(854, 616, 156, 222)
(222, 156, 3)
(854, 619, 156, 222)
(222, 156, 3)
(854, 619, 156, 222)
(222, 156, 3)
(854, 622, 156, 222)
(222, 156, 3)
(854, 622, 156, 222)
(222, 156, 3)
(854, 625, 156, 222)
(222, 156, 3)
(854, 625, 156, 222)
(222, 156, 3)
(854, 628, 156, 222)
(222, 156, 3)
(854, 628, 156, 222)
(222, 156, 3)
(854, 631, 156, 222)
(222, 156, 3)
(854, 631, 156, 222)
(222, 156, 3)
(854, 634, 156, 222)
(222, 156, 3)
(854, 634, 156, 222)

In [8]:
N = 1
list(range(-N, N+1, 1))

[-1, 0, 1]